<a href="https://colab.research.google.com/github/Ramakant90/AGI/blob/main/RamakantBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import json

intents = {
  "intents": [
    {"tag": "greeting",
     "patterns": ["Hi", "Hello", "Hey", "Namaste", "What's up?"],
     "responses": ["Hello!", "Namaste!", "Hi there! How can I help?"]
    },
    {"tag": "bye",
     "patterns": ["Bye", "See you", "Goodbye"],
     "responses": ["Alvida!", "Bye!", "See you soon!"]
    },
    {"tag": "thanks",
     "patterns": ["Thanks", "Thank you", "Shukriya"],
     "responses": ["You're welcome!", "Koi baat nahi!", "Happy to help!"]
    },
    {"tag": "age",
     "patterns": ["What is your age?", "How old are you?"],
     "responses": ["Main AI hoon, umar ka koi matlab nahi 😄"]
    }
  ]
}

with open("intents.json", "w") as f:
    json.dump(intents, f, indent=4)

print("✅ intents.json file created!")

✅ intents.json file created!


In [ ]:
!pip install torch torchvision nltk

import json
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import nltk
from nltk.stem.porter import PorterStemmer

nltk.download('punkt')
nltk.download('punkt_tab') # Download the missing resource
stemmer = PorterStemmer()

# 🔹 Helper functions
def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

# 🔹 Load dataset
with open("intents.json", "r") as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []

for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)
        all_words.extend(w)
        xy.append((w, tag))

ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words]
all_words = sorted(set(all_words))
tags = sorted(set(tags))

X_train = []
y_train = []

for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    y_train.append(tags.index(tag))

X_train = np.array(X_train)
y_train = np.array(y_train)

# 🔹 Model Design
class ChatBotModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChatBotModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

# 🔹 Training
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
model = ChatBotModel(input_size, hidden_size, output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1000):
    X = torch.from_numpy(X_train)
    y = torch.from_numpy(y_train).long()

    output = model(X)
    loss = criterion(output, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 200 == 0:
        print(f'Epoch [{epoch+1}/1000], Loss: {loss.item():.4f}')

print("✅ Training complete. Saving model...")

data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

torch.save(data, "chatbot_data.pth")
print("💾 Model saved as chatbot_data.pth")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Epoch [200/1000], Loss: 0.7011
Epoch [400/1000], Loss: 0.0833
Epoch [600/1000], Loss: 0.0163
Epoch [800/1000], Loss: 0.0067
Epoch [1000/1000], Loss: 0.0036
✅ Training complete. Saving model...
💾 Model saved as chatbot_data.pth


In [ ]:

import torch
import random
import json
import nltk
from nltk.stem.porter import PorterStemmer
import numpy as np

stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):
    tokenized_sentence = [stem(w) for w in tokenized_sentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0
    return bag

# 🔹 Load model
FILE = "chatbot_data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data["all_words"]
tags = data["tags"]
model_state = data["model_state"]

class ChatBotModel(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(ChatBotModel, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, output_size)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

model = ChatBotModel(input_size, hidden_size, output_size)
model.load_state_dict(model_state)
model.eval()

with open("intents.json", "r") as f:
    intents = json.load(f)

bot_name = "RamakantBot"
print(f"{bot_name} tayyar hai! Type 'quit' to exit.\n")

while True:
    sentence = input("You: ")
    if sentence.lower() == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = torch.from_numpy(X).float()
    output = model(X)
    _, predicted = torch.max(output, dim=0)
    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=0)
    prob = probs[predicted.item()]

    if prob.item() > 0.7:
        for intent in intents["intents"]:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: Mujhe samajh nahi aaya 😅")

RamakantBot tayyar hai! Type 'quit' to exit.

RamakantBot: Hello!
RamakantBot: Mujhe samajh nahi aaya 😅
RamakantBot: Mujhe samajh nahi aaya 😅


In [ ]:
from google.colab import drive
drive.mount('/content/drive')